In [24]:
from xml.etree import cElementTree as ET

In [25]:
tree = ET.parse('desc2021.xml')
root = tree.getroot()

In [26]:
descName = []
descUI = []
descTree = []
conceptName = []
conceptUI = []
term = []


#loop through DescriptorRecords
for DescriptorRecord in root.findall('DescriptorRecord'):
    
    #loop through DescriptorNames (should only be 1 per DescriptorRecord)
    r = []
    for DescriptorName in DescriptorRecord.findall('DescriptorName'):
        r.append(DescriptorName.find('String').text)
    descName.append(r)
    
    #loop through DescriptorUIs (should only be 1 per DescriptorRecord)
    r = []
    for DescriptorUI in DescriptorRecord.findall('DescriptorUI'):
        r.append(DescriptorUI.text)
    descUI.append(r)
    
    #loop thourgh Descriptor Tree assignments (can be multiple)
    r = []
    for TreeNumberList in DescriptorRecord.findall('TreeNumberList'):
        for tree in TreeNumberList.findall('TreeNumber'):
            r.append(tree.text)
    descTree.append(r)
    
    #loop through Concepts in each DescriptorRecord (can be multiple)
    name_r = []
    ui_r = []
    term_r = []
    for ConceptList in DescriptorRecord.findall('ConceptList'):
        for Concept in ConceptList.findall('Concept'):
            #select preferred concept
            if Concept.attrib['PreferredConceptYN'] == 'Y':
                #loop through name and id for preferred concept (should only be 1 each)
                for name in Concept.findall('ConceptName'):
                    name_r.append(name.find('String').text)
                for ui in Concept.findall('ConceptUI'):
                    ui_r.append(ui.text)
                #loop through terms for preferred concept
                for TermList in Concept.findall('TermList'):
                    for Term in TermList.findall('Term'):
                        #label term as preferred or not
                        if Term.attrib['ConceptPreferredTermYN'] == 'Y':
                            term_r.append({'termName':Term.find('String').text, 'preferred': 'Y'})
                        else:
                            term_r.append({'termName':Term.find('String').text, 'preferred': 'N'})
            else:
                continue
    conceptName.append(name_r)
    conceptUI.append(ui_r)
    term.append(term_r)

In [27]:
import pandas as pd
df = pd.DataFrame({'descName':descName,'descUI':descUI,'tree':descTree,'conceptName':conceptName,'conceptUI':conceptUI, 'term':term})

In [28]:
df.head()

,descName,descUI,tree,conceptName,conceptUI,term
0,[Calcimycin],[D000001],[D03.633.100.221.173],[Calcimycin],[M0000001],"[{'termName': 'Calcimycin', 'preferred': 'Y'}]"
1,[Temefos],[D000002],"[D02.705.400.625.800, D02.705.539.345.800, D02...",[Temefos],[M0000002],"[{'termName': 'Temefos', 'preferred': 'Y'}, {'..."
2,[Abattoirs],[D000003],"[J01.576.423.200.700.100, J03.540.020]",[Abattoirs],[M0000003],"[{'termName': 'Abattoirs', 'preferred': 'Y'}, ..."
3,[Abbreviations as Topic],[D000004],[L01.559.598.400.556.131],[Abbreviations as Topic],[M0000004],"[{'termName': 'Abbreviations as Topic', 'prefe..."
4,[Abdomen],[D000005],[A01.923.047],[Abdomen],[M0000005],"[{'termName': 'Abdomen', 'preferred': 'Y'}, {'..."


In [29]:
df.shape

(29917, 6)

In [30]:
df.loc[:,'descName'] = [x[0] for x in df['descName']]
df.loc[:,'descUI'] = [x[0] for x in df['descUI']]
df.loc[:,'conceptName'] = [x[0] for x in df['conceptName']]
df.loc[:,'conceptUI'] = [x[0] for x in df['conceptUI']]

In [31]:
df = df.explode('tree')

In [33]:
df.loc[:,'tree_parent'] = [str(x)[:3] for x in df['tree']]

In [34]:
df.loc[:,'tree_length'] = [len(str(x)) for x in df['tree']]

In [36]:
disease = df[df['tree_parent'].str.startswith('C')]

In [37]:
mental_health = df[df['tree_parent'] == 'F03']

In [38]:
mesh = pd.concat([disease, mental_health])

In [41]:
def Mode(l):
    return max(set(l), key=l.count)

In [42]:
def Mini(l):
    return min(set(l), key=l.count)

In [43]:
def descui_to_single_category(mesh):

    r = {'descUI':[],'group':[]}

    for ui in mesh['descUI'].unique():
        df_ = mesh[mesh['descUI'] == ui]
        counts = df_['tree_parent'].value_counts().to_dict()
        if len(counts) == 1:
            r['descUI'].append(ui)
            r['group'].append([x for x in counts.keys()][0])
        else:
            l = df_['tree_parent'].tolist()
            mode = Mode(l)
            mini = Mini(l)
            if mode == mini:
                #then pick the broadest category
                min_tree = min(df_['tree_length'].tolist())
                r['descUI'].append(ui)
                r['group'].append(df_[df_['tree_length'] == min_tree]['tree_parent'].tolist()[0])
            else:
                r['descUI'].append(ui)
                r['group'].append(mode)
    
    return pd.DataFrame(r)

In [44]:
single_category = descui_to_single_category(mesh)

In [45]:
df = pd.merge(mesh, single_category, on='descUI')

In [46]:
df = df.explode('term')

In [47]:
df.loc[:,'termName'] = [x['termName'] for x in df['term']]
df.loc[:,'termPref'] = [x['preferred'] for x in df['term']]
del df['term']

In [51]:
df[df['tree'] == 'F03.700.750']

,descName,descUI,tree,conceptName,conceptUI,tree_parent,tree_length,group,termName,termPref
12394,Schizophrenia,D012559,F03.700.750,Schizophrenia,M0019489,F03,11,F03,Schizophrenia,Y
12394,Schizophrenia,D012559,F03.700.750,Schizophrenia,M0019489,F03,11,F03,Schizophrenias,N
12394,Schizophrenia,D012559,F03.700.750,Schizophrenia,M0019489,F03,11,F03,Schizophrenic Disorders,N
12394,Schizophrenia,D012559,F03.700.750,Schizophrenia,M0019489,F03,11,F03,"Disorder, Schizophrenic",N
12394,Schizophrenia,D012559,F03.700.750,Schizophrenia,M0019489,F03,11,F03,"Disorders, Schizophrenic",N
12394,Schizophrenia,D012559,F03.700.750,Schizophrenia,M0019489,F03,11,F03,Schizophrenic Disorder,N
12394,Schizophrenia,D012559,F03.700.750,Schizophrenia,M0019489,F03,11,F03,Dementia Praecox,N


In [52]:
def FormatTerm(df):
    r = []
    for term in df['termName']:
        term = term.lower()
        split = term.split(',')
        if len(split) > 1:
            #reverse list
            new_item = ' '.join([x for x in reversed(split)])
            r.append(new_item.lstrip())
        else:
            r.append(term.lstrip())
    return r

In [53]:
df.loc[:,'term_clean'] = FormatTerm(df)

In [54]:
df.loc[:,'termName'] = [x.lower().replace(',','') for x in df['termName']]

In [56]:
df.to_csv('mesh_with_mental_health.csv', index=False)

In [55]:
df['group'].value_counts()

C10    18149
C01    11789
C04    10895
C16    10486
C18     9855
C14     6749
C05     4583
C23     3741
C06     3581
C15     3332
C08     2935
C11     2755
C17     2730
C07     2088
C12     1984
C13     1770
C26     1741
C19     1506
C20     1318
F03     1277
C25      832
C22      819
C09      659
C24      123
C21       57
Name: group, dtype: int64